# word2vec

## Google Tutorial
https://www.tensorflow.org/tutorials/word2vec/

## This file has slightly modified Google code
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/word2vec/word2vec_basic.py


In [2]:
# modied from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/word2vec/word2vec_basic.py

# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#         http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange    # pylint: disable=redefined-builtin
import tensorflow as tf

# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'


def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
                'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

# jwm: get file without script: 
# wget http://mattmahoney.net/dc/text8.zip -O data/text8.zip
filename = maybe_download('text8.zip', 31344016)


# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

words = read_data(filename)
print('Data size', len(words))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000


def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0    # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words    # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0


# Step 3: Function to generate a training batch for the skip-gram model.
# jwm: for each word, get all of the words around it
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1    # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window    # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        # jwm: as you append, others fall off. [1 2 3].append(4) --> [2 3 4]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # jwm: batch[23] is a word, and labels[23,0] is a word nearby
    return batch, labels

# jwm: print out what a batch looks like
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],
                '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.

batch_size = 128 # updates the same set of batch 
embedding_size = 128    # Dimension of the embedding vector.
skip_window = 1             # How many words to consider left and right.
num_skips = 2                 # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16         # Random set of words to evaluate similarity on.
valid_window = 100    # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64        # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
                tf.truncated_normal([vocabulary_size, embedding_size],
                                                        stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
            tf.nn.nce_loss(weights=nce_weights,
                                         biases=nce_biases,
                                         labels=train_labels,
                                         inputs=embed,
                                         num_sampled=num_sampled,
                                         num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
                batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8    # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()
    
    # jwm: save embeddings to see on tensorboard
    saver = tf.train.Saver()
    LOG_DIR = '/Users/shambhavi.srivastava/tensorflow_dataday2018/tensorlog/'
    saver.save(session, LOG_DIR+'model.ckpt', step)


Found and verified text8.zip
Data size 17005207
Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
3081 originated -> 5234 anarchism
3081 originated -> 12 as
12 as -> 3081 originated
12 as -> 6 a
6 a -> 12 as
6 a -> 195 term
195 term -> 2 of
195 term -> 6 a
Initialized
Average loss at step  0 :  268.5105895996094
Nearest to that: locks, probing, husk, connie, swear, vorkosigan, affordable, librarian,
Nearest to was: weakest, mri, americanization, epicurus, accumulating, bactrian, evaporated, commodores,
Nearest to eight: sympathies, fujiko, cts, qa, ls, monotonous, somaliland, anybody,
Nearest to some: laches, cleanly, nothing, abate, unstable, distraught, grits, bpm,
Nearest to nine: subcode, correspond, dictatorship, yourself, mwali, spiny, unwillingly, gleaned,
Nearest to at: t

Average loss at step  52000 :  5.178177094936371
Average loss at step  54000 :  5.1203557230234145
Average loss at step  56000 :  5.068719156265259
Average loss at step  58000 :  5.115915567278862
Average loss at step  60000 :  4.920337778687477
Nearest to that: which, this, but, sotho, cebus, thibetanus, agouti, microcebus,
Nearest to was: is, has, had, were, by, been, dasyprocta, be,
Nearest to eight: nine, seven, six, four, five, three, zero, callithrix,
Nearest to some: many, these, all, their, the, thibetanus, other, most,
Nearest to nine: eight, six, seven, four, five, zero, three, agouti,
Nearest to at: in, through, on, microcebus, cdp, from, abakan, thibetanus,
Nearest to who: he, they, which, chef, that, there, she, capuchin,
Nearest to or: and, microcebus, agouti, callithrix, tamarin, dasyprocta, victoriae, bebox,
Nearest to other: many, some, victoriae, farther, orbital, dasyprocta, mathbf, strategist,
Nearest to if: where, but, became, when, abakan, is, although, dasyprocta

In [2]:
# There is some tricky code in generate_batch
import collections
buffer = collections.deque(maxlen=3)
buffer.append(1)
buffer.append(2)
buffer.append(3)
print(buffer[1])
buffer.append(4)
print(buffer[1])



2
3


In [5]:
#******************************************************************
# Begin TensorBoard embedding code
# code from https://www.tensorflow.org/how_tos/embedding_viz/
#******************************************************************
# jwm: save the names of the words to a file
with open(LOG_DIR+'metadata.tsv','w') as f:
    for i in range(vocabulary_size):
        f.write(reverse_dictionary[i]+'\n')

# jwm: code from https://www.tensorflow.org/how_tos/embedding_viz/
from tensorflow.contrib.tensorboard.plugins import projector

# Use the same LOG_DIR where you stored your checkpoint.
#summary_writer = tf.train.SummaryWriter(LOG_DIR) # depricated
summary_writer = tf.summary.FileWriter(LOG_DIR)

# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embeddings.name # jwm: changed to match our embeddings

# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = LOG_DIR+'metadata.tsv'

# Saves a configuration file that TensorBoard will read during startup.
projector.visualize_embeddings(summary_writer, config)